In [1]:
pip install pillow lxml

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\otman\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
def process_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find('filename').text
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    image_info = {
        'filename': filename,
        'width': width,
        'height': height,
        'objects': []
    }

    for obj in root.iter('object'):
        obj_dict = {
            'name': obj.find('name').text,
            'coordinates': [int(obj.find('bndbox').find('xmin').text), 
                            int(obj.find('bndbox').find('ymin').text)]
        }
        image_info['objects'].append(obj_dict)

    return image_info

In [4]:
def augment_image(image, rotation_datagen, scale_factor, num_to_generate, image_info, output_dir):
    i = 0
    original_size = image.size
    scaled_size = (int(original_size[0] * scale_factor), int(original_size[1] * scale_factor))
    scaled_image = image.resize(scaled_size, Image.ANTIALIAS)

    for batch in rotation_datagen.flow(np.expand_dims(np.array(scaled_image), 0), batch_size=1):
        batch_image = Image.fromarray(batch[0].astype('uint8'), 'RGB')
        new_filename = f"aug_{i}_{image_info['filename']}"
        batch_image.save(os.path.join(output_dir, new_filename))
        
        # Update the dictionary with new filename and (for simplicity) same coordinates
        new_image_info = image_info.copy()
        new_image_info['filename'] = new_filename
        all_data.append(new_image_info)
        
        i += 1
        if i >= num_to_generate:
            break

In [5]:
rotation_datagen = ImageDataGenerator(rotation_range=20)  # Adjust as needed for rotation

# Scale factor for zooming out
scale_factor = 0.8 

In [9]:
image_dir = r'C:\Users\otman\Downloads\est\image'
annotation_dir = r'C:\Users\otman\Downloads\est\annotation'
output_dir = r'C:\Users\otman\Downloads\est\augmented_images'
output_dir_csv = r'C:\Users\otman\Downloads\est\augmented_csv'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_dir_csv, exist_ok=True)

In [12]:
all_data = []
num_augmentations_per_image = 4  # Set so that total is around 200
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_dir, filename)
        xml_path = os.path.join(annotation_dir, filename.replace('.jpg', '.xml'))

        # Load and process the image and XML
        image = Image.open(image_path)
        image_info = process_xml(xml_path)

        # Augment image and update dictionary
        augment_image(image, rotation_datagen, scale_factor, num_augmentations_per_image, image_info, output_dir)
# Convert to DataFrame and save as CSV
df = pd.DataFrame(all_data)
df.to_csv(r'C:\Users\otman\Downloads\est\augmented_csv\dict1.csv', index=False)

C:\Users\otman\AppData\Local\Temp\ipykernel_35656\4110955332.py:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  scaled_image = image.resize(scaled_size, Image.ANTIALIAS)


In [2]:


def augment_image(image, datagen, num_to_generate, image_info, output_dir):
    i = 0
    for batch in datagen.flow(np.expand_dims(np.array(image), 0), batch_size=1):
        batch_image = Image.fromarray(batch[0].astype('uint8'), 'RGB')
        new_filename = f"aug_{i}_{image_info['filename']}"
        batch_image.save(os.path.join(output_dir, new_filename))
        
        # Update the dictionary with new filename and (for simplicity) same coordinates
        new_image_info = image_info.copy()
        new_image_info['filename'] = new_filename
        all_data.append(new_image_info)
        
        i += 1
        if i >= num_to_generate:
            break

# Create an ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,  # Adjust as needed for rotation
    zoom_range=[0.5, 1.0]  # Adjust as needed for zooming out
)

# Paths


# Process and augment images
all_data = []
num_augmentations_per_image = 4  # Set so that total is around 200
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_dir, filename)
        xml_path = os.path.join(annotation_dir, filename.replace('.jpg', '.xml'))

        # Load and process the image and XML
        image = Image.open(image_path)
        image_info = process_xml(xml_path)

        # Augment image and update dictionary
        augment_image(image, datagen, num_augmentations_per_image, image_info, output_dir)

# Convert to DataFrame and save as CSV
df = pd.DataFrame(all_data)
df.to_csv('/kaggle/working/dict.csv', index=False)

In [36]:
from PIL import Image



def rotate_point(x, y, angle, cx, cy):
        angle_rad = np.radians(angle)
        x_new = np.cos(angle_rad) * (x - cx) - np.sin(angle_rad) * (y - cy) + cx
        y_new = np.sin(angle_rad) * (x - cx) + np.cos(angle_rad) * (y - cy) + cy
        return x_new, y_new

def update_coordinates(obj_dict, angle, scale_factor, image_size):
    # Unpack original image dimensions and bounding box coordinates
    original_width, original_height = image_size
    xmin, ymin = obj_dict['coordinates']

    # Calculate the center of the image
    center_x, center_y = original_width / 2, original_height / 2

    # Scale the coordinates
    xmin_scaled, ymin_scaled = xmin * scale_factor, ymin * scale_factor

    # Rotate the scaled coordinates
    xmin_rotated, ymin_rotated = rotate_point(xmin_scaled, ymin_scaled, angle, center_x * scale_factor, center_y * scale_factor)

    # Update object dictionary with new coordinates
    obj_dict['coordinates'] = [int(xmin_rotated), int(ymin_rotated)]

    return obj_dict

NameError: name 'xmin' is not defined

In [14]:
def augment_and_save(image_path, xml_path, output_dir, angle, scale_factor, counter):
    # Load image and parse XML
    image = Image.open(image_path)
    image_info = process_xml(xml_path)

    # Perform scaling and rotation
    scaled_image = image.resize((int(image.width * scale_factor), int(image.height * scale_factor)), Image.ANTIALIAS)
    rotated_image = scaled_image.rotate(angle, expand=True)

    # Update XML data with transformed coordinates
    for obj in image_info['objects']:
        update_coordinates(obj, angle, scale_factor, (image.width, image.height))

    # Save the transformed image
    new_filename = f"aug_{counter}_{image_info['filename']}"
    rotated_image.save(os.path.join(output_dir, new_filename))

    # Update image_info with new filename
    image_info['filename'] = new_filename
    return image_info

In [15]:
angles = [10, -10]  # Example rotation angles
scale_factors = [0.8, 0.9]

In [17]:
all_data = []
counter = 0
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg') and counter < num_augmentations_per_image:
        image_path = os.path.join(image_dir, filename)
        xml_path = os.path.join(annotation_dir, filename.replace('.jpg', '.xml'))

        for angle in angles:
            for scale_factor in scale_factors:
                augmented_data = augment_and_save(image_path, xml_path, output_dir, angle, scale_factor, counter)
                all_data.append(augmented_data)
                counter += 1
                if counter >= num_augmentations_per_image:
                    break
            if counter >= num_augmentations_per_image:
                break

# Convert to DataFrame and save as CSV
df = pd.DataFrame(all_data)
df.to_csv(r'C:\Users\otman\Downloads\est\augmented_csv\dict4.csv', index=False)

C:\Users\otman\AppData\Local\Temp\ipykernel_35656\3437959015.py:7: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  scaled_image = image.resize((int(image.width * scale_factor), int(image.height * scale_factor)), Image.ANTIALIAS)


In [46]:
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw

def get_annotations_from_df(df, image_filename):
    """Get annotations for a specific image from the DataFrame."""
    annotations = []
    row = df[df['filename'] == image_filename].iloc[0]
    for obj in row['objects']:
        xmin, ymin = obj['coordinates']
        xmax, ymax = xmin, ymin  # Adjust if you have xmax, ymax
        annotations.append((xmin, ymin, xmax, ymax))
    return annotations

def draw_annotations(image_path, annotations):
    """Draw annotations on the image."""
    with Image.open(image_path) as img:
        draw = ImageDraw.Draw(img)
        for ann in annotations:
            # Draw a rectangle for each annotation
            draw.rectangle(ann, outline="red", width=3)
        img.show()

# Example usage
image_path = r'C:\Users\otman\Downloads\est\augmented_images\aug_0_0.jpg'  # Replace with your image path

draw_annotations(image_path, annotations)

annotations = get_annotations_from_df(df, 'aug_0_0.jpg')
draw_annotations(image_path, annotations)

ValueError: too many values to unpack (expected 2)

In [32]:
df.head()

,filename,width,height,objects
0,aug_0_0.jpg,448,448,"[{'name': 'coin_1', 'coordinates': [84, 39]}, ..."
1,aug_1_0.jpg,448,448,"[{'name': 'coin_1', 'coordinates': [94, 44]}, ..."
2,aug_2_0.jpg,448,448,"[{'name': 'coin_1', 'coordinates': [42, 80]}, ..."
3,aug_3_0.jpg,448,448,"[{'name': 'coin_1', 'coordinates': [47, 90]}, ..."


In [34]:
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw

def load_annotation(xml_file):
    """Load the annotation from the XML file."""
    tree = ET.parse(xml_file)
    root = tree.getroot()

    annotations = []
    for obj in root.iter('object'):
        xmin = int(obj.find('bndbox').find('xmin').text)
        ymin = int(obj.find('bndbox').find('ymin').text)
        xmax = int(obj.find('bndbox').find('xmax').text)
        ymax = int(obj.find('bndbox').find('ymax').text)
        annotations.append((xmin, ymin, xmax, ymax))
    return annotations

def draw_annotations(image_path, annotations):
    """Draw annotations on the image."""
    with Image.open(image_path) as img:
        draw = ImageDraw.Draw(img)
        for ann in annotations:
            # Draw a rectangle for each annotation
            draw.rectangle(ann, outline="red", width=2)
        img.show()

# Example usage
image_path = r'C:\Users\otman\Downloads\est\image\0.jpg'  # Replace with your image path
xml_path = r'C:\Users\otman\Downloads\est\annotation\0.xml'  # Replace with your XML path



annotations = load_annotation(xml_path)
draw_annotations(image_path, annotations)

In [47]:
def rotate_point(x, y, angle, cx, cy):
    angle_rad = np.radians(angle)
    x_new = np.cos(angle_rad) * (x - cx) - np.sin(angle_rad) * (y - cy) + cx
    y_new = np.sin(angle_rad) * (x - cx) + np.cos(angle_rad) * (y - cy) + cy
    return x_new, y_new

def get_rotated_bbox(xmin, ymin, xmax, ymax, angle, cx, cy):
    corners = [(xmin, ymin), (xmax, ymin), (xmin, ymax), (xmax, ymax)]
    rotated_corners = [rotate_point(x, y, angle, cx, cy) for x, y in corners]

    x_coords, y_coords = zip(*rotated_corners)
    xmin_new, xmax_new = min(x_coords), max(x_coords)
    ymin_new, ymax_new = min(y_coords), max(y_coords)

    return [xmin_new, ymin_new, xmax_new, ymax_new]

def process_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find('filename').text
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    image_info = {
        'filename': filename,
        'width': width,
        'height': height,
        'objects': []
    }

    for obj in root.iter('object'):
        obj_dict = {
            'name': obj.find('name').text,
            'coordinates': [int(obj.find('bndbox').find('xmin').text), 
                            int(obj.find('bndbox').find('ymin').text)]
        }
        image_info['objects'].append(obj_dict)

    return image_info

def augment_and_save(image_path, xml_path, output_dir, angle, scale_factor, counter):
    image = Image.open(image_path)
    image_info = process_xml(xml_path)

    scaled_image = image.resize((int(image.width * scale_factor), int(image.height * scale_factor)), Image.ANTIALIAS)
    rotated_image = scaled_image.rotate(angle, expand=True)

    image_center = (scaled_image.width / 2, scaled_image.height / 2)
    for obj in image_info['objects']:
        xmin, ymin = obj['coordinates']
        xmax, ymax = xmin, ymin  # Assuming point-based bbox
        obj['coordinates'] = get_rotated_bbox(xmin, ymin, xmax, ymax, angle, *image_center)

    new_filename = f"aug_{counter}_{image_info['filename']}"
    rotated_image.save(os.path.join(output_dir, new_filename))

    image_info['filename'] = new_filename
    return image_info

# Set up your parameters and paths
angles = [10, -10]  # Example rotation angles
scale_factors = [0.8, 0.9]  # Example scale factors
num_augmentations_per_image = 4

all_data = []
counter = 0
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg') and counter < num_augmentations_per_image:
        image_path = os.path.join(image_dir, filename)
        xml_path = os.path.join(annotation_dir, filename.replace('.jpg', '.xml'))

        for angle in angles:
            for scale_factor in scale_factors:
                augmented_data = augment_and_save(image_path, xml_path, output_dir, angle, scale_factor, counter)
                all_data.append(augmented_data)
                counter += 1
                if counter >= num_augmentations_per_image:
                    break
            if counter >= num_augmentations_per_image:
                break
                
df = pd.DataFrame(all_data)
df.to_csv(r'C:\Users\otman\Downloads\est\augmented_csv\dict6.csv', index=False)

C:\Users\otman\AppData\Local\Temp\ipykernel_35656\2274490722.py:47: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  scaled_image = image.resize((int(image.width * scale_factor), int(image.height * scale_factor)), Image.ANTIALIAS)


In [50]:
import os
import numpy as np
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET

def rotate_point(x, y, angle, cx, cy):
    angle_rad = np.radians(angle)
    x_new = np.cos(angle_rad) * (x - cx) - np.sin(angle_rad) * (y - cy) + cx
    y_new = np.sin(angle_rad) * (x - cx) + np.cos(angle_rad) * (y - cy) + cy
    return x_new, y_new

def get_transformed_bbox(bbox, angle, scale_factor, image_center):
    xmin, ymin, xmax, ymax = bbox
    xmin, ymin, xmax, ymax = xmin * scale_factor, ymin * scale_factor, xmax * scale_factor, ymax * scale_factor

    corners = [(xmin, ymin), (xmax, ymin), (xmin, ymax), (xmax, ymax)]
    rotated_corners = [rotate_point(x, y, angle, *image_center) for x, y in corners]

    xs, ys = zip(*rotated_corners)
    xmin_new, ymin_new = min(xs), min(ys)
    xmax_new, ymax_new = max(xs), max(ys)

    return [xmin_new, ymin_new, xmax_new, ymax_new]

def process_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find('filename').text
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    image_info = {
        'filename': filename,
        'width': width,
        'height': height,
        'objects': []
    }

    for obj in root.iter('object'):
        xmin = int(obj.find('bndbox').find('xmin').text)
        ymin = int(obj.find('bndbox').find('ymin').text)
        xmax = int(obj.find('bndbox').find('xmax').text)
        ymax = int(obj.find('bndbox').find('ymax').text)
        image_info['objects'].append({'xmin': xmin, 'ymin': ymin, 'xmax': xmax, 'ymax': ymax})

    return image_info

def augment_and_save(image_path, xml_path, output_dir, angle, scale_factor, counter):
    image = Image.open(image_path)
    image_info = process_xml(xml_path)

    scaled_image = image.resize((int(image.width * scale_factor), int(image.height * scale_factor)), Image.ANTIALIAS)
    rotated_image = scaled_image.rotate(angle, expand=True)

    image_center = (scaled_image.width / 2, scaled_image.height / 2)
    draw = ImageDraw.Draw(rotated_image)

    for obj in image_info['objects']:
        bbox = get_rotated_bbox(obj['xmin'], obj['ymin'], obj['xmax'], obj['ymax'], angle, scale_factor, image_center)
        draw.rectangle(bbox, outline="red", width=2)

    new_filename = f"aug_{counter}_{image_info['filename']}"
    rotated_image.save(os.path.join(output_dir, new_filename))

    return new_filename
angles = [10, -10]  # Example rotation angles
scale_factors = [0.8, 0.9]  # Example scale factors
num_total_images = 50  # Total number of images you want to generate

# Process and augment images
counter = 0
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg') and counter < num_total_images:
        image_path = os.path.join(image_dir, filename)
        xml_path = os.path.join(annotation_dir, filename.replace('.jpg', '.xml'))

        for angle in angles:
            for scale_factor in scale_factors:
                if counter >= num_total_images:
                    break
                new_filename = augment_and_save(image_path, xml_path, output_dir, angle, scale_factor, counter)
                counter += 1

# Check if the required number of images are generated
print(f"Generated {counter} augmented images with bounding boxes.")


C:\Users\otman\AppData\Local\Temp\ipykernel_35656\2107064367.py:54: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  scaled_image = image.resize((int(image.width * scale_factor), int(image.height * scale_factor)), Image.ANTIALIAS)


Generated 50 augmented images with bounding boxes.


In [211]:
import os
import cv2
import xml.etree.ElementTree as ET

def process_xml(xml_file):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        filename = root.find('filename').text
        size = root.find('size')
        width = int(size.find('width').text)
        height = int(size.find('height').text)

        image_info = {
            'filename': filename,
            'width': width,
            'height': height,
            'objects': []
        }

        for obj in root.iter('object'):
            xmin = int(float(obj.find('bndbox').find('xmin').text))
            ymin = int(float(obj.find('bndbox').find('ymin').text))
            image_info['objects'].append({'name': obj.find('name').text, 'coordinates': [xmin, ymin]})

        return image_info

    except ET.ParseError as e:
        print(f"Error parsing file {xml_file}: {e}")
        return None

def flip_horizontal(bbox, image_width):
    xmin, ymin = bbox
    return [image_width - xmin, ymin]

def flip_vertical(bbox, image_height):
    xmin, ymin = bbox
    return [xmin, image_height - ymin]

def draw_bounding_box(image, bbox):
    xmin, ymin = bbox
    cv2.circle(image, (int(xmin), int(ymin)), radius=2, color=(0, 255, 0), thickness=4)


def rotate_point(x, y, angle, cx, cy):
    angle_rad = np.radians(angle)
    x_new = np.cos(angle_rad) * (x - cx) - np.sin(angle_rad) * (y - cy) + cx
    y_new = np.sin(angle_rad) * (x - cx) + np.cos(angle_rad) * (y - cy) + cy
    return x_new, y_new

def rotate_bbox(bbox, angle, image_center):
    xmin, ymin = bbox
    rotated = rotate_point(xmin, ymin, angle, *image_center)
    return rotated
image_directory = r'C:\Users\otman\Downloads\est\image'
xml_directory = r'C:\Users\otman\Downloads\est\annotation'
output_directory = r'C:\Users\otman\Downloads\est\transformed_images'
os.makedirs(output_directory, exist_ok=True)
output_directory_drawn = r'C:\Users\otman\Downloads\est\transformed_images_drawn'
os.makedirs(output_directory_drawn, exist_ok=True)
xml_output_path=r'C:\Users\otman\Downloads\est\transformed_xml'
os.makedirs(xml_output_path, exist_ok=True)

In [116]:
def apply_transformations_and_save(image_dir, xml_dir, output_dir, output_dir_drawn, transformations):
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(image_dir, filename)
            xml_path = os.path.join(xml_dir, filename.replace('.jpg', '.xml'))
            data = process_xml(xml_path)

            if data is None:
                continue

            image = cv2.imread(image_path)
            
            if image is None:
                print(f"Unable to read image: {filename}")
                continue
            image_center = (image.shape[1] // 2, image.shape[0] // 2)
            for transformation in transformations:
                if transformation['type'] == 'flip_horizontal':
                    image = cv2.flip(image, 1)
                    suffix = '_horizontal'
                    update_function = flip_horizontal
                elif transformation['type'] == 'flip_vertical':
                    image = cv2.flip(image, 0)
                    suffix = '_vertical'
                    update_function = flip_vertical
                elif transformation['type'] == 'rotate':
                    angle = transformation.get('angle', 90)
                    M = cv2.getRotationMatrix2D(image_center, angle, 1.0)
                    image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
                    suffix = f'_rotated_{angle}'
                    update_function = lambda bbox, img_shape: rotate_bbox(bbox, angle, image_center)

                new_filename = os.path.splitext(filename)[0] + suffix + os.path.splitext(filename)[1]
                cv2.imwrite(os.path.join(output_dir, new_filename), image)

                # Draw bounding boxes and save the drawn image
                image_with_bbox = image.copy()
                for obj in data['objects']:
                    obj['coordinates'] = update_function(obj['coordinates'], image.shape[1 if suffix == '_horizontal' else 0])
                    draw_bounding_box(image_with_bbox, obj['coordinates'])

                cv2.imwrite(os.path.join(output_dir_drawn, new_filename), image_with_bbox)

                data['filename'] = new_filename

# Example usage
image_directory = r'C:\Users\otman\Downloads\est\image'
xml_directory = r'C:\Users\otman\Downloads\est\annotation'
output_directory = r'C:\Users\otman\Downloads\est\transformed_images'
os.makedirs(output_directory, exist_ok=True)
output_directory_drawn = r'C:\Users\otman\Downloads\est\transformed_images_drawn'
os.makedirs(output_directory_drawn, exist_ok=True)
xml_output_path=r'C:\Users\otman\Downloads\est\transformed_xml'
os.makedirs(xml_output_path, exist_ok=True)
transformations = [{'type': 'flip_horizontal'}, {'type': 'flip_vertical'}]

apply_transformations_and_save(image_directory, xml_directory, output_directory, output_directory_drawn, transformations)

In [160]:
def save_xml(data, xml_output_path, bbox_size=(20, 20)):
    root = ET.Element("annotation")

    # Add elements to the XML as per your schema
    filename = ET.SubElement(root, "filename")
    filename.text = data['filename']

    size = ET.SubElement(root, "size")
    width = ET.SubElement(size, "width")
    width.text = str(data['width'])
    height = ET.SubElement(size, "height")
    height.text = str(data['height'])
    depth = ET.SubElement(size, "depth")
    depth.text = "3"  # Assuming all images are RGB

    for obj in data['objects']:
        obj_elem = ET.SubElement(root, "object")
        name = ET.SubElement(obj_elem, "name")
        name.text = obj['name']

        bndbox = ET.SubElement(obj_elem, "bndbox")
        xmin = ET.SubElement(bndbox, "xmin")
        xmin.text = str(obj['coordinates'][0])
        ymin = ET.SubElement(bndbox, "ymin")
        ymin.text = str(obj['coordinates'][1])
        xmax = ET.SubElement(bndbox, "xmax")
        xmax.text = str(obj['coordinates'][0] + bbox_size[0])  # Calculate xmax
        ymax = ET.SubElement(bndbox, "ymax")
        ymax.text = str(obj['coordinates'][1] + bbox_size[1])   # Assuming ymax is stored

    tree = ET.ElementTree(root)
    tree.write(xml_output_path)


In [162]:
def apply_horizontal_and_save(image_path, xml_path, output_dir, output_dir_drawn, filename):
    data = process_xml(xml_path)
    if data is None:
        print(f"Unable to process XML: {filename}")
        return

    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to read image: {filename}")
        return

    # Apply horizontal flip
    image_flipped = cv2.flip(image, 1)
    for obj in data['objects']:
        obj['coordinates'] = flip_horizontal(obj['coordinates'], image.shape[1])

    # Save the flipped image
    new_filename = os.path.splitext(filename)[0] + '_horizontal' + os.path.splitext(filename)[1]
    cv2.imwrite(os.path.join(output_dir, new_filename), image_flipped)

    # Draw bounding boxes and save the image
    image_with_bbox = image_flipped.copy()
    for obj in data['objects']:
        draw_bounding_box(image_with_bbox, obj['coordinates'])
    cv2.imwrite(os.path.join(output_dir_drawn, new_filename), image_with_bbox)
    
    data['filename'] = new_filename

    # Save the updated XML
    xml_output_path = os.path.join(xml_directory, new_filename.replace('.jpg', '.xml'))
    save_xml(data, xml_output_path)
    
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_directory, filename)
        xml_path = os.path.join(xml_directory, filename.replace('.jpg', '.xml'))

        # Apply transformations
        apply_horizontal_and_save(image_path, xml_path, output_directory, output_directory_drawn, filename)


In [212]:
def apply_vertical_and_save(image_path, xml_path, output_dir, output_dir_drawn, filename):
    data = process_xml(xml_path)
    if data is None:
        print(f"Unable to process XML: {filename}")
        return

    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to read image: {filename}")
        return

    # Apply vertical flip
    image_flipped = cv2.flip(image, 0)
    for obj in data['objects']:
        obj['coordinates'] = flip_vertical(obj['coordinates'], image.shape[0])

    # Save the flipped image
    new_filename = os.path.splitext(filename)[0] + '_vertical' + os.path.splitext(filename)[1]
    cv2.imwrite(os.path.join(output_dir, new_filename), image_flipped)

    # Draw bounding boxes and save the image
    image_with_bbox = image_flipped.copy()
    for obj in data['objects']:
        draw_bounding_box(image_with_bbox, obj['coordinates'])
    cv2.imwrite(os.path.join(output_dir_drawn, new_filename), image_with_bbox)
    data['filename'] = new_filename

    # Save the updated XML
    xml_output_path = os.path.join(xml_directory, new_filename.replace('.jpg', '.xml'))
    save_xml(data, xml_output_path)
    
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_directory, filename)
        xml_path = os.path.join(xml_directory, filename.replace('.jpg', '.xml'))

        # Apply transformations
        apply_vertical_and_save(image_path, xml_path, output_directory, output_directory_drawn, filename)


In [213]:
def apply_rotation_and_save(image_path, xml_path, output_dir, output_dir_drawn, filename, angle):
    data = process_xml(xml_path)
    if data is None:
        print(f"Unable to process XML: {filename}")
        return

    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to read image: {filename}")
        return

    image_center = (image.shape[1] // 2, image.shape[0] // 2)
    M = cv2.getRotationMatrix2D(image_center, angle, 1)
    image_rotated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    for obj in data['objects']:
        obj['coordinates'] = rotate_bbox(obj['coordinates'], angle, image_center)

    # Save the rotated image
    new_filename = os.path.splitext(filename)[0] + f'_rotated_{angle}' + os.path.splitext(filename)[1]
    cv2.imwrite(os.path.join(output_dir, new_filename), image_rotated)

    # Draw bounding boxes and save the image
    image_with_bbox = image_rotated.copy()
    for obj in data['objects']:
        draw_bounding_box(image_with_bbox, obj['coordinates'])
    cv2.imwrite(os.path.join(output_dir_drawn, new_filename), image_with_bbox)
    
    data['filename'] = new_filename

    # Save the updated XML
    xml_output_path = os.path.join(xml_directory, new_filename.replace('.jpg', '.xml'))
    save_xml(data, xml_output_path)

angle = 180  # Rotation angle

for filename in os.listdir(image_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_directory, filename)
        xml_path = os.path.join(xml_directory, filename.replace('.jpg', '.xml'))
        apply_rotation_and_save(image_path, xml_path, output_directory, output_directory_drawn, filename, angle)

In [172]:
def scale_image_and_boxes(image, bbox_list, scale_factor, bbox_size=(20, 20)):
    new_size = (int(image.shape[1] * scale_factor), int(image.shape[0] * scale_factor))
    scaled_image = cv2.resize(image, new_size, interpolation=cv2.INTER_LINEAR)

    scaled_bbox_list = []
    for bbox in bbox_list:
        xmin, ymin = bbox
        xmax, ymax = xmin + bbox_size[0], ymin + bbox_size[1]
        scaled_bbox = [int(xmin * scale_factor), int(ymin * scale_factor),
                       int(xmax * scale_factor), int(ymax * scale_factor)]
        scaled_bbox_list.append(scaled_bbox)

    return scaled_image, scaled_bbox_list

def apply_scaling_and_save(image_path, xml_path, output_dir, output_dir_drawn, filename, scale_factor):
    data = process_xml(xml_path)
    if data is None:
        print(f"Unable to process XML: {filename}")
        return

    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to read image: {filename}")
        return

    scaled_image, scaled_bbox_list = scale_image_and_boxes(image, [obj['coordinates'] for obj in data['objects']], scale_factor)

    # Update the filename and bounding boxes in the data dictionary
    data['filename'] = os.path.splitext(filename)[0] + f'_scaled_{scale_factor}' + os.path.splitext(filename)[1]
    for obj, new_bbox in zip(data['objects'], scaled_bbox_list):
        obj['coordinates'] = new_bbox

    # Save the scaled image and XML
    new_filename = data['filename']
    cv2.imwrite(os.path.join(output_dir, new_filename), scaled_image)
    save_xml(data, os.path.join(xml_directory, new_filename.replace('.jpg', '.xml')))

    # Draw bounding boxes and save the image
    image_with_bbox = scaled_image.copy()
    for new_bbox in scaled_bbox_list:
        draw_bounding_box(image_with_bbox, new_bbox)
    cv2.imwrite(os.path.join(output_dir_drawn, new_filename), image_with_bbox)
def draw_bounding_box(image, bbox):
    xmin, ymin, xmax, ymax = bbox

    # Drawing a rectangle around the bounding box
    cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)

# Example usage
scale_factor = 0.5  # Example scale factor
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_directory, filename)
        xml_path = os.path.join(xml_directory, filename.replace('.jpg', '.xml'))
        apply_scaling_and_save(image_path, xml_path, output_directory, output_directory_drawn, filename, scale_factor)


In [175]:
def draw_bounding_box(image, bbox):
    xmin, ymin = bbox
    cv2.circle(image, (int(xmin), int(ymin)), radius=2, color=(0, 255, 0), thickness=4)

def add_gaussian_noise(image):
    row, col, ch = image.shape
    mean = 0
    var = 0.1
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy_image = image + gauss
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

def apply_noise_and_save(image_path, xml_path, output_dir, output_dir_drawn, filename):
    data = process_xml(xml_path)
    if data is None:
        print(f"Unable to process XML: {filename}")
        return

    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to read image: {filename}")
        return

    noisy_image = add_gaussian_noise(image)

    # Save the noisy image
    new_filename = os.path.splitext(filename)[0] + '_noisy' + os.path.splitext(filename)[1]
    cv2.imwrite(os.path.join(output_dir, new_filename), noisy_image)

    # Update the filename in the data dictionary and save the XML
    data['filename'] = new_filename
    xml_output_path = os.path.join(xml_directory, new_filename.replace('.jpg', '.xml'))
    save_xml(data, xml_output_path)

    # Draw bounding boxes on the noisy image for visualization
    image_with_bbox = noisy_image.copy()
    for obj in data['objects']:
        draw_bounding_box(image_with_bbox, obj['coordinates'])
    cv2.imwrite(os.path.join(output_dir_drawn, new_filename), image_with_bbox)
    
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_directory, filename)
        xml_path = os.path.join(xml_directory, filename.replace('.jpg', '.xml'))
        apply_noise_and_save(image_path, xml_path, output_directory, output_directory_drawn, filename)

In [179]:
def adjust_brightness(image, brightness_factor):
    # Ensure the brightness factor is within a reasonable range
    brightness_factor = max(min(brightness_factor, 2.0), 0)  # Restricting between 0 and 2

    adjusted_image = image * brightness_factor
    adjusted_image = np.clip(adjusted_image, 0, 255)  # Ensuring the values are within 0-255
    return adjusted_image.astype(np.uint8)

def apply_brightness_and_save(image_path, xml_path, output_dir, output_dir_drawn, filename, brightness_factor):
    data = process_xml(xml_path)
    if data is None:
        print(f"Unable to process XML: {filename}")
        return

    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to read image: {filename}")
        return

    bright_image = adjust_brightness(image.astype(np.float32)/255, brightness_factor)

    # Save the brightened image
    new_filename = os.path.splitext(filename)[0] + f'_bright_{brightness_factor}' + os.path.splitext(filename)[1]
    cv2.imwrite(os.path.join(output_dir, new_filename), (bright_image * 255).astype(np.uint8))

    # Draw bounding boxes and save the image
    image_with_bbox = (bright_image * 255).astype(np.uint8).copy()
    for obj in data['objects']:
        draw_bounding_box(image_with_bbox, obj['coordinates'])
    cv2.imwrite(os.path.join(output_dir_drawn, new_filename), image_with_bbox)

    # Update the filename in the data dictionary and save the XML
    data['filename'] = new_filename
    save_xml(data, os.path.join(xml_directory, new_filename.replace('.jpg', '.xml')))

# Example usage
brightness_factor = 1.5  # Example brightness factor
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_directory, filename)
        xml_path = os.path.join(xml_directory, filename.replace('.jpg', '.xml'))
        apply_brightness_and_save(image_path, xml_path, output_directory, output_directory_drawn, filename, brightness_factor)



In [113]:

def apply_transformations_and_save(image_dir, xml_dir, output_dir, output_dir_drawn, transformations):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(output_dir_drawn):
        os.makedirs(output_dir_drawn)

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(image_dir, filename)
            xml_path = os.path.join(xml_dir, filename.replace('.jpg', '.xml'))
            data = process_xml(xml_path)

            if data is None:
                continue

            image = cv2.imread(image_path)
            if image is None:
                print(f"Unable to read image: {filename}")
                continue

            image_center = (image.shape[1] // 2, image.shape[0] // 2)

            for transformation in transformations:
                image_transformed = image.copy()

                if transformation['type'] == 'flip_horizontal':
                    image_transformed = cv2.flip(image_transformed, 1)
                    suffix = '_horizontal'
                    for obj in data['objects']:
                        obj['coordinates'] = flip_horizontal(obj['coordinates'], image.shape[1])
                elif transformation['type'] == 'flip_vertical':
                    image_transformed = cv2.flip(image_transformed, 0)
                    suffix = '_vertical'
                    for obj in data['objects']:
                        # Update the bbox for the flipped image
                        obj['coordinates'] = flip_vertical(obj['coordinates'], image.shape[0])
                elif transformation['type'] == 'rotate':
                    angle = transformation['angle']
                    M = cv2.getRotationMatrix2D(image_center, angle, 1)
                    image_transformed = cv2.warpAffine(image_transformed, M, (image.shape[1], image.shape[0]))
                    suffix = f'_rotated_{angle}'
                    for obj in data['objects']:
                        obj['coordinates'] = rotate_bbox(obj['coordinates'], angle, image_center)

                new_filename = os.path.splitext(filename)[0] + suffix + os.path.splitext(filename)[1]
                cv2.imwrite(os.path.join(output_dir, new_filename), image_transformed)

                # Draw bounding boxes and save the drawn image
                image_with_bbox = image_transformed.copy()
                for obj in data['objects']:
                    draw_bounding_box(image_with_bbox, obj['coordinates'])

                cv2.imwrite(os.path.join(output_dir_drawn, new_filename), image_with_bbox)

                data['filename'] = new_filename

# Example usage
image_directory = r'C:\Users\otman\Downloads\est\image'
xml_directory = r'C:\Users\otman\Downloads\est\annotation'
output_directory = r'C:\Users\otman\Downloads\est\transformed_images'
output_directory_drawn = r'C:\Users\otman\Downloads\est\transformed_images_drawn'
transformations = [
    {'type': 'flip_horizontal'}, 
    {'type': 'flip_vertical'},
    {'type': 'rotate', 'angle': 90}  # Example rotation angle
]

apply_transformations_and_save(image_directory, xml_directory, output_directory, output_directory_drawn, transformations)

In [109]:
def apply_transformations_and_save(image_dir, xml_dir, output_dir, output_dir_drawn, transformations):
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(image_dir, filename)
            xml_path = os.path.join(xml_dir, filename.replace('.jpg', '.xml'))
            data = process_xml(xml_path)

            if data is None:
                continue

            image = cv2.imread(image_path)
            if image is None:
                print(f"Unable to read image: {filename}")
                continue

            for transformation in transformations:
                transformed_image = image.copy()

                if transformation['type'] == 'flip_horizontal':
                    image = cv2.flip(image, 1)
                    suffix = '_horizontal'
                    update_function = flip_horizontal
                elif transformation['type'] == 'flip_vertical':
                    image = cv2.flip(image, 0)
                    suffix = '_vertical'
                    update_function = flip_vertical

                new_filename = os.path.splitext(filename)[0] + suffix + os.path.splitext(filename)[1]
                cv2.imwrite(os.path.join(output_dir, new_filename), image)

                # Draw bounding boxes and save the drawn image
                image_with_bbox = image.copy()
                for obj in data['objects']:
                    obj['coordinates'] = update_function(obj['coordinates'], image.shape[1 if suffix == '_horizontal' else 0])
                    draw_bounding_box(image_with_bbox, obj['coordinates'])

                cv2.imwrite(os.path.join(output_dir_drawn, new_filename), image_with_bbox)

                data['filename'] = new_filename

In [117]:
transformations = [
    {'type': 'flip_horizontal'}, 
    {'type': 'flip_vertical'},
    {'type': 'rotate', 'angle': 90}  # Example rotation angle
]

apply_transformations_and_save(image_directory, xml_directory, output_directory, output_directory_drawn, transformations)

In [164]:
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_directory, filename)
        xml_path = os.path.join(xml_directory, filename.replace('.jpg', '.xml'))

        # Apply transformations
        apply_horizontal_and_save(image_path, xml_path, output_directory, output_directory_drawn, filename)
        apply_vertical_and_save(image_path, xml_path, output_directory, output_directory_drawn, filename)
        apply_rotation_and_save(image_path, xml_path, output_directory, output_directory_drawn, filename, 90)

TypeError: rotate_bbox() takes 3 positional arguments but 4 were given

In [165]:
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_directory, filename)
        xml_path = os.path.join(xml_directory, filename.replace('.jpg', '.xml'))
        apply_rotation_and_save(image_path, xml_path, output_directory, output_directory_drawn, filename, 350)

TypeError: rotate_bbox() takes 3 positional arguments but 4 were given

In [217]:
def load_data(image_dir, xml_dir):
    images = []
    labels = []

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(image_dir, filename)
            xml_path = os.path.join(xml_dir, filename.replace('.jpg', '.xml'))

            # Read and preprocess the image
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))  # Resize for the model
            images.append(image)

            # Parse the XML to get the points
            tree = ET.parse(xml_path)
            root = tree.getroot()
            points = []
            for member in root.findall('object'):
                # Assuming each 'object' element contains one point with (xmin, ymin, xmax, ymax)
                xmin = int(float(member.find('bndbox').find('xmin').text))
                ymin = int(float(member.find('bndbox').find('ymin').text))
                # Optionally, calculate the center or any specific point from (xmin, ymin, xmax, ymax)
                points.extend([xmin, ymin])  # Adjust according to your requirement
            labels.append(points)

    return np.array(images), np.array(labels)


image_directory = r'C:\Users\otman\Downloads\est\transformed_images'
xml_directory = r'C:\Users\otman\Downloads\est\annotation'

train_images, train_labels = load_data(image_directory, xml_directory)


In [225]:
from sklearn.model_selection import train_test_split

def normalize_labels(labels, image_size=(448, 448)):
    # Normalize labels to be between 0 and 1
    normalized_labels = []
    for label in labels:
        normalized_label = []
        for i in range(0, len(label), 2):
            normalized_label.extend([label[i] / image_size[0], label[i+1] / image_size[1]])
        normalized_labels.append(normalized_label)
    return np.array(normalized_labels)

normalized_train_labels = normalize_labels(train_labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_images, normalized_train_labels, test_size=0.2, random_state=42)

In [233]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import Concatenate, Activation, Flatten, Dense
def create_model(input_shape):
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Middle
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    
    # Decoder
    up4 = UpSampling2D(size=(2, 2))(conv3)
    up4 = Conv2D(128, 2, activation='relu', padding='same')(up4)
    concat4 = Concatenate(axis=-1)([conv2, up4])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(concat4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    
    up5 = UpSampling2D(size=(2, 2))(conv4)
    up5 = Conv2D(64, 2, activation='relu', padding='same')(up5)
    concat5 = Concatenate(axis=-1)([conv1, up5])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(concat5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    # Output layer for point prediction
    flattened = Flatten()(conv5)
    dense_layer = Dense(128, activation='relu')(flattened)
    output_layer = Dense(8, activation='sigmoid')(dense_layer)  # 8 values for 4 points (x, y)

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model


In [ ]:
model = create_model((224,224,3))
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
9/9 [==============================] - 835s 88s/step - loss: 0.3096 - val_loss: 0.2711
Epoch 2/10
6/9 [===================>..........] - ETA: 4:13 - loss: 0.3015

In [229]:
test_loss = model.evaluate(X_test, y_test)
print("Test loss:", test_loss)

3/3 [==============================] - 0s 83ms/step - loss: 0.2134
Test loss: 0.2133806347846985


In [227]:
model.save(r'C:\Users\otman\Downloads\est\model.h5')

C:\Users\otman\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [228]:

# Load and preprocess the image
new_image_path = r'C:\Users\otman\Downloads\est\image\0.jpg'  # Replace with your image path

model = load_model(r'C:\Users\otman\Downloads\est\model.h5')  # Adjust the path to your model

def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

import cv2
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model


def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    preprocessed_image = cv2.resize(image, (224, 224))  # Resize as per model's input shape
    return preprocessed_image, image  # Return both preprocessed and original image

def draw_points_on_image(image, points):
    for i in range(0, len(points), 2):
        x, y = int(points[i]), int(points[i+1])
        cv2.circle(image, (x, y), radius=5, color=(0, 255, 0), thickness=-1)
    return image

predicted_directory = r'C:\Users\otman\Downloads\est\predicted'
create_directory(predicted_directory)

preprocessed_image, original_image = load_and_preprocess_image(new_image_path)

# Predict points using the model
predictions = model.predict(np.expand_dims(preprocessed_image / 255.0, axis=0))
predicted_points = predictions[0]

# Assuming points are normalized, rescale them back
predicted_points = np.array([predicted_points[i] * original_image.shape[1] if i % 2 == 0 else predicted_points[i] * original_image.shape[0] for i in range(len(predicted_points))])

# Draw the points on the original image
output_image = draw_points_on_image(original_image.copy(), predicted_points)

print("Predicted points:", predicted_points)
# Save the image with drawn points
output_filename = os.path.basename(new_image_path).replace('.jpg', '_predicted.jpg')
cv2.imwrite(os.path.join(predicted_directory, output_filename), cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR))

1/1 [==============================] - 0s 73ms/step
Predicted points: [ 2.47182477 -0.02691932  6.50476754 -1.60085137  3.08626473 -2.6095067
 -0.69678748  3.64408207]


True